In [1]:
import pandas as pd



In [2]:
from NFLVersReader.src.nflverse_clean.pbp_job import perform_workflow

results = perform_workflow()

2023-06-30 15:47:37,050 - INFO - Impute columns to 0
2023-06-30 15:47:37,100 - INFO - impute non binary pbp columns ...
2023-06-30 15:47:37,149 - INFO - Impute columns to 0
2023-06-30 15:47:37,219 - INFO - Impute columns to 0:00
2023-06-30 15:47:37,360 - INFO - Impute columns to NA
2023-06-30 15:47:38,050 - INFO - moving play_id to play_counter, and creating a joinable play_id key
2023-06-30 15:47:38,107 - INFO - Conform key actions like pass, rush, kickoff, etc. and add a single category field called actions... 
2023-06-30 15:47:38,657 - INFO - Validate actions dimension ...
2023-06-30 15:47:38,681 - INFO - Creating new drive dimension...
2023-06-30 15:47:38,687 - INFO - Validate drive_df dimension ...
2023-06-30 15:47:38,717 - INFO - Creating new situations dimension...
2023-06-30 15:47:38,725 - INFO - Validate situation_df dimension ...
2023-06-30 15:47:38,750 - INFO - Creating new metrics dimension...
2023-06-30 15:47:38,756 - INFO - Validate play_metrics_df dimension ...
2023-06-3

In [3]:
results.keys()

dict_keys(['play_actions', 'game_drive', 'play_situations', 'play_metrics', 'play_identities', 'player_events', 'game_info', 'schema', 'player_participation', 'pbp_participation', 'player_stats', 'players', 'player_injuries'])

In [4]:
from NFLVersReader.src.nflverse_clean.pbp_fact import clean_columns

pbp_df = pd.read_csv("../../output/playbyplay_2021.csv", low_memory=False)
players_df = results['player_participation']
clean_columns(pbp_df)

2023-06-30 15:47:43,890 - INFO - Impute columns to 0
INFO:pbp_logger:Impute columns to 0
2023-06-30 15:47:43,914 - INFO - impute non binary pbp columns ...
INFO:pbp_logger:impute non binary pbp columns ...
2023-06-30 15:47:43,956 - INFO - Impute columns to 0
INFO:pbp_logger:Impute columns to 0
2023-06-30 15:47:44,022 - INFO - Impute columns to 0:00
INFO:pbp_logger:Impute columns to 0:00
2023-06-30 15:47:44,161 - INFO - Impute columns to NA
INFO:pbp_logger:Impute columns to NA
2023-06-30 15:47:44,878 - INFO - moving play_id to play_counter, and creating a joinable play_id key
INFO:pbp_logger:moving play_id to play_counter, and creating a joinable play_id key


In [47]:
offense_teams = pbp_df[['play_id', 'posteam']].drop_duplicates()
offense_teams.rename(columns={'posteam': 'team'}, inplace=True)
offense_teams.shape

(50712, 2)

In [48]:
offense_teams.head()

,play_id,team
0,2021_01_ARI_TEN_1,NA
1,2021_01_ARI_TEN_40,TEN
2,2021_01_ARI_TEN_55,TEN
3,2021_01_ARI_TEN_76,TEN
4,2021_01_ARI_TEN_100,TEN


In [40]:
defense_teams = pbp_df[['play_id', 'defteam']].drop_duplicates()
defense_teams.rename(columns={'defteam': 'team'}, inplace=True)
defense_teams.shape

(50712, 2)

In [41]:
offense_players = players_df[players_df['lineup'] == 'offense']
defense_players = players_df[players_df['lineup'] == 'defense']

In [42]:
offense_players.shape

(504618, 4)

In [43]:
defense_players.shape

(504531, 4)

In [44]:
offense_merged_df = pd.merge(offense_players, offense_teams, on='play_id')
offense_merged_df.shape

(504618, 5)

In [45]:
defense_merged_df = pd.merge(defense_players, defense_teams, on='play_id')
defense_merged_df.shape

(504531, 5)

In [46]:
defense_merged_df.head()

,game_id,play_id,player_id,lineup,team
0,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0036643,defense,DAL
1,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0033595,defense,DAL
2,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0034515,defense,DAL
3,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0032637,defense,DAL
4,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0030456,defense,DAL


In [49]:
merged_df = pd.concat([offense_merged_df, defense_merged_df])

In [51]:
merged_df.head()

,game_id,play_id,player_id,lineup,team
0,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0033083,offense,TB
1,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0033512,offense,TB
2,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0034674,offense,TB
3,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0036882,offense,TB
4,2021_01_DAL_TB,2021_01_DAL_TB_40,00-0034373,offense,TB
